# Extracción de información

In [ ]:
import spacy
import pandas as pd
from spacy import displacy

nlp = spacy.load("es_core_news_sm")

In [ ]:
nlp.component_names

## Named Entity Recognition

In [ ]:
doc = nlp("El gran escritor Miguel de Cervantes nació en Alcalá de Henares")

In [ ]:
print([(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
displacy.render(doc, style='ent', jupyter=True)

In [ ]:
entidades = [e for e in doc.ents]
entidades

In [ ]:
type(entidades[0])

In [ ]:
entidades[0].start

In [ ]:
entidades[0].end

In [ ]:
doc[3:6]

In [ ]:
entidades[0].label_

In [ ]:
datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

Podemos usar la detección de entidades de tipo `PER` para enmascarar nombres propios en un texto

In [ ]:
texto = "Al detenido Alejandro García se le acusa de matar a Juan Pérez"

In [ ]:
doc = nlp(texto)
datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

In [ ]:
nlp.add_pipe("merge_entities")

In [ ]:
nlp.component_names

In [ ]:
doc = nlp(texto)
datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

In [ ]:
[t.text if not t.ent_type_=='PER' else '_PERSONA_' for t in doc]

### Creación de nuevas entidades en spaCy
Podemos crear entidades nuevas con el componente `EntityRuler` del pipeline (https://spacy.io/api/entityruler)

In [ ]:
#Definimos "escritor" como profesión
ruler = nlp.add_pipe("entity_ruler")
patterns = [{"label": "PROFESION", "pattern": [{"LOWER": "escritor"}]}]
ruler.add_patterns(patterns)

doc = nlp("El gran escritor Miguel de Cervantes nació en Alcalá de Henares")
displacy.render(doc, style='ent', jupyter=True)

In [ ]:
doc = nlp("Miguel de Cervantes era Escritor")
displacy.render(doc, style='ent', jupyter=True)

In [ ]:
datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

Podemos añadir nuevas profesiones

In [ ]:
patterns = [{"label": "PROFESION", "pattern": [{"LOWER": "matador"},{"LOWER": "de"},{"LOWER": "toros"}]}]
ruler.add_patterns(patterns)

doc = nlp("El padre de Miguel Bosé era el matador de toros Luis Miguel Dominguín")
print([(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

In [ ]:
doc = nlp("Luis Miguel era matador de toros y Cervantes era escritor")
datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

Como el proceso `entity_ruler` está detrás del `merge_entities` en el *pipeline* de `spaCy` no une las profesiones

In [ ]:
nlp.component_names

In [ ]:
nlp.remove_pipe('merge_entities')

In [ ]:
doc = nlp("Luis Miguel era matador de toros y Cervantes era escritor")

datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

In [ ]:
nlp.add_pipe("merge_entities")
nlp.component_names

In [ ]:
doc = nlp("Luis Miguel era matador de toros y Cervantes era escritor")

datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

In [ ]:
patterns = [{"label": "ANIMAL", "pattern": [{"LEMMA": "gato"},{"LEMMA": "montés", 'OP': '?'}]},
           {"label": "ANIMAL", "pattern": [{"LEMMA": "perro"}]}]
ruler.add_patterns(patterns)

doc = nlp("Los gatos y perros son animales de compañía.")
print([(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'lemma': t.lemma_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

In [ ]:
doc = nlp("El gato montés es más grande que los gatos callejeros")

datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'lemma': t.lemma_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

In [ ]:
print([(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
doc = nlp("El escritor Miguel de Cervantes tenía un gato llamado Juan")
print([(ent.text, ent.label_) for ent in doc.ents])

## Entity linking
Usamos la API de DBPedia Spotlight (https://www.dbpedia-spotlight.org/api)

In [ ]:
import requests
from IPython.core.display import display, HTML
# An API Error Exception
class APIError(Exception):
    def __init__(self, status):
            self.status = status
    def __str__(self):
            return "APIError: status={}".format(self.status)
      
# Base URL for Spotlight API
base_url = "http://api.dbpedia-spotlight.org/en/annotate"
# Parameters 
# 'text' - text to be annotated 
# 'confidence' -   confidence score for linking
text = "The Space Shuttle was a partially reusable low Earth orbital spacecraft system operated from April 12, 1981, to July 21, 2011, by the National Aeronautics and Space Administration in the United States. Launched from the Kennedy Space Center in Florida, five Space Shuttle orbiter vehicles flew on a total of 135 missions during 30 years."

params = {"text": text, "confidence": 0.5}
# Response content type
headers = {'accept': 'text/html'}
# GET Request
res = requests.get(base_url, params=params, headers=headers)
if res.status_code != 200:
    # Something went wrong
    raise APIError(res.status_code)
# Display the result as HTML in Jupyter Notebook
display(HTML(res.text))

In [ ]:
res.headers

In [ ]:
print(res.text)

Podemos hacer la petición en formato JSON

In [ ]:
headers = {'accept': 'application/json'}
# GET Request
res = requests.get(base_url, params=params, headers=headers)
if res.status_code != 200:
    # Something went wrong
    raise APIError(res.status_code)
# Display the result as HTML in Jupyter Notebook
respuesta = res.json()
respuesta

La respuesta JSON se puede iterar como un diccionario de claves-valores:

In [ ]:
for key, value in respuesta.items():
    print(f"{key} : {value}\n")

Además podemos acceder directamente a una clave o a una clave anidada:

In [ ]:
respuesta['@text']

In [ ]:
type(respuesta['Resources'])

In [ ]:
respuesta['Resources'][0]['@URI']

In [ ]:
    
# Base URL for Spotlight API
base_url = "http://api.dbpedia-spotlight.org/es/annotate"
# Parameters 
# 'text' - text to be annotated 
# 'confidence' -   confidence score for linking
with open('articulo.txt', 'r') as f:
    texto = f.read()
    
params = {"text": texto, "confidence": 0.5}
# Response content type
headers = {'accept': 'application/json'}
# GET Request
res = requests.get(base_url, params=params, headers=headers)
if res.status_code != 200:
    # Something went wrong
    raise APIError(res.status_code)
# Display the result as HTML in Jupyter Notebook
respuesta = res.json()
respuesta

In [ ]:
for r in respuesta['Resources']:
    print(r['@URI'])